# 条件表达式

In [ ]:
%cd ../..
import set_env

In [2]:
import numpy as np
import tvm
from tvm import te

在 numpy 中使用 {func}`~numpy.where` 处理数组的条件表达式：

In [3]:
a = np.arange(10)
np.where(a < 5, a, 10*a)

array([ 0,  1,  2,  3,  4, 50, 60, 70, 80, 90])

In [4]:
a = np.array([[0, 1, 2],
              [0, 2, 4],
              [0, 3, 6]])
np.where(a < 4, a, -1)  # -1 被广播

array([[ 0,  1,  2],
       [ 0,  2, -1],
       [ 0,  3, -1]])

在 TVM 中使用 `if_then_else` 实现它。与 {func}`~numpy.where` 类似，它接受三个参数，第一个是条件，如果为真返回第二个参数，否则返回第三个参数。

下面以实现上三角矩阵为例：

In [5]:
n, m = te.var('n'), te.var('m')
A = te.placeholder((m, n))
B = te.compute(A.shape,
               lambda i, j: te.if_then_else(i >= j, A[i, j], 0.0))
te_func = te.create_prim_func([A, B])
te_func.show()
mod = tvm.build(te_func, target="llvm")

In [6]:
a_np = np.arange(1, 13, dtype='float32').reshape((3, 4))
b_np = np.tril(a_np)
b_np

array([[ 1.,  0.,  0.,  0.],
       [ 5.,  6.,  0.,  0.],
       [ 9., 10., 11.,  0.]], dtype=float32)

In [7]:
a_nd = tvm.nd.array(a_np)
b_nd = tvm.nd.array(np.empty_like(a_np))
mod(a_nd, b_nd)
b_nd

<tvm.nd.NDArray shape=(3, 4), cpu(0)>
array([[ 1.,  0.,  0.,  0.],
       [ 5.,  6.,  0.,  0.],
       [ 9., 10., 11.,  0.]], dtype=float32)